# SignalDigger
SignalDigger是digger模块中的一个核心类，使用SigalDigger可以分析股票单因子各项表现

*** 步骤 ***
1. 实例化SignalDigger-指定因子绩效表现的输出路径和输出格式
2. 因子数据预处理-通常跟dataview结合使用
2. 进行因子研究和分析

# step 1 实例化SignalDigger-指定因子绩效表现的输出路径和输出格式

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import jaqs_fxdayu
jaqs_fxdayu.patch_all()

In [7]:
from jaqs.research import SignalDigger

# step 1：实例化SignalDigger 通过output_folder和output_format指定因子绩效表现的输出路径和输出格式
sd = SignalDigger(output_folder=".", output_format='pdf')

# step 2 因子数据预处理

## process_signal_before_analysis

- ` jaqs.research.signaldigger.digger.SignalDigger.process_signal_before_analysis(*args, **kwargs) `

**简要描述：**

- 将因子数据处理成signal_data--包含了因子及相关收益、quantile分组、涨跌空间等评价维度的综合数据(pandas.MultiIndex)

**参数：**


|参数名|必选|类型|说明|
|:----    |:---|:----- |-----   |
|signal|是  |pandas.DataFrame|因子值,日期为索引，股票品种为columns|
|price |是，price与ret二选一  |pandas.DataFrame|因子涉及到的股票的价格数据，用于作为进出场价用于计算收益,日期为索引，股票品种为columns|
|ret |是，price与ret二选一  |pandas.DataFrame| 因子涉及到的股票的持有期收益，日期为索引，股票品种为columns|
|benchmark_price | 否  |pandas.DataFrame or pandas.Series|基准价格，日期为索引。在price参数不为空的情况下，该参数生效，用于计算因子涉及到的股票的持有期**相对收益**--相对基准。默认为空，为空时计算的收益为**绝对收益**。|
|high |否  |pandas.DataFrame|因子涉及到的股票的最高价数据,用于计算持有期潜在最大上涨收益,日期为索引，股票品种为columns,默认为空|
|low |否  |pandas.DataFrame|因子涉及到的股票的最低价数据,用于计算持有期潜在最大下跌收益,日期为索引，股票品种为columns,默认为空|
|group |否  |pandas.DataFrame|因子涉及到的股票的分组(行业分类),日期为索引，股票品种为columns,默认为空|
|period |否  |int|持有周期,默认为5,即持有5天|
|n_quantiles |否  |int|根据每日因子值的大小分成n_quantiles组,默认为5,即将因子每天分成5组|
|mask |否  |pandas.DataFrame|一张由bool值组成的表格,日期为索引，股票品种为columns，表示在做因子分析时是否要对某期的某个品种过滤。对应位置为True则**过滤**（剔除）——不纳入因子分析考虑。默认为空，不执行过滤操作|
|can_enter |否  |pandas.DataFrame|一张由bool值组成的表格,日期为索引，股票品种为columns，表示某期的某个品种是否可以买入(进场)。对应位置为True则可以买入。默认为空，任何时间任何品种均可买入|
|can_exit |否  |pandas.DataFrame|一张由bool值组成的表格,日期为索引，股票品种为columns，表示某期的某个品种是否可以卖出(出场)。对应位置为True则可以卖出。默认为空，任何时间任何品种均可卖出|
|forward |否  |bool|收益对齐方式,forward=True则在当期因子下对齐下一期实现的收益；forward=False则在当期实现收益下对齐上一期的因子值。默认为True|
|commission |否 |float|手续费比例,每次换仓收取的手续费百分比,默认为万分之八0.0008|


**示例：**

In [8]:
import numpy as np
from jaqs.data import DataView

# 加载dataview数据集
dv = DataView()
dataview_folder = './data'
dv.load_dataview(dataview_folder)

# 定义信号过滤条件-非指数成分
df_index_member = dv.get_ts('index_member')
mask = dv.get_ts('index_member') == 0  # 定义信号过滤条件-非指数成分

# 定义可买入卖出条件——未停牌、未涨跌停
trade_status = dv.get_ts('trade_status')
can_trade = ~(trade_status == u'停牌') # 未停牌
# 涨停
up_limit = dv.add_formula('up_limit', '(open - Delay(close, 1)) / Delay(close, 1) > 0.095', is_quarterly=False)
# 跌停
down_limit = dv.add_formula('down_limit', '(open - Delay(close, 1)) / Delay(close, 1) < -0.095', is_quarterly=False)
can_enter = np.logical_and(up_limit < 1, can_trade)  # 未涨停未停牌
can_exit = np.logical_and(down_limit < 1, can_trade)  # 未跌停未停牌

Dataview loaded successfully.


In [9]:
sd.process_signal_before_analysis(signal=dv.get_ts("pe"),
                                   price=dv.get_ts("close_adj"),
                                   high=dv.get_ts("high_adj"),
                                   low=dv.get_ts("low_adj"),
                                   group=dv.get_ts("sw1"),
                                   n_quantiles=5,
                                   mask=mask,
                                   can_enter=can_enter,
                                   can_exit=can_exit,
                                   period=5,
                                   benchmark_price=dv.data_benchmark,
                                   forward=True,
                                   commission=0.0008
                                   )

Nan Data Count (should be zero) : 0;  Percentage of effective data: 85%


# step 3 进行因子研究和分析

## signal_data

- ` jaqs.research.signaldigger.digger.SignalDigger.signal_data `

**简要描述：**

- process_signal_before_analysis方法处理后的signal_data--包含了因子及相关收益、quantile分组、涨跌空间等评价维度的综合数据(pandas.MultiIndex)
- 需要执行process_signal_before_analysis后该属性才有值

**返回：**

trade_date+symbol为MultiIndex,columns为signal(因子)、return(持有期相对/绝对收益)、upside_ret(持有期潜在最大上涨收益)、downside_ret(持有期潜在最大下跌收益)、group(分组/行业分类)、quantile(按因子值分组)

**示例：**

In [10]:
sd.signal_data.head()

signal    return  upside_ret  downside_ret   group  \
trade_date symbol                                                           
20170503   000001.SZ   6.7925 -0.005637   -0.003045     -0.042326  480000   
           000002.SZ  10.0821  0.011225    0.016697     -0.029432  430000   
           000008.SZ  42.9544 -0.049408    0.000463     -0.092972  640000   
           000009.SZ  79.4778 -0.069822    0.009714     -0.095426  510000   
           000027.SZ  20.4542 -0.019517    0.009404     -0.041616  410000   

                      quantile  
trade_date symbol               
20170503   000001.SZ         1  
           000002.SZ         1  
           000008.SZ         4  
           000009.SZ         5  
           000027.SZ         2

## benchmark_ret

- ` jaqs.research.signaldigger.digger.SignalDigger.benchmark_ret `

**简要描述：**

- 基准收益
- 需要执行process_signal_before_analysis且传入了benchmark_price后该属性才有值

**示例：**

In [11]:
sd.benchmark_ret.tail()

,close
trade_date,
20170925,-0.006597
20170926,-0.002958
20170927,-0.005527
20170928,-0.003981
20170929,-0.000320


## ret

- ` jaqs.research.signaldigger.digger.SignalDigger.ret `

**简要描述：**

- 因子相关股票池的收益信息
- 需要执行process_signal_before_analysis且传入了benchmark_price后该属性才有值

**返回：**

由股票池各股票的return、upside_ret、downside_ret所组成的字典(dict)

**示例：**

In [12]:
print(sd.ret.keys())
sd.ret["return"].head()

dict_keys(['return', 'upside_ret', 'downside_ret'])


symbol,000001.SZ,000002.SZ,000008.SZ,000009.SZ,000027.SZ,000039.SZ,000060.SZ,000061.SZ,000063.SZ,000069.SZ,...,601988.SH,601989.SH,601992.SH,601997.SH,601998.SH,603000.SH,603160.SH,603858.SH,603885.SH,603993.SH
trade_date,,,,,,,,,,,,,,,,,,,,,
20170502,-0.012748,-0.009400,-0.060645,-0.042922,-0.015162,-0.032807,0.005199,0.000423,-0.006402,-0.011290,...,0.023618,-0.049392,-0.113737,0.001664,0.007408,-0.016581,0.013146,-0.019445,-0.012823,-0.022670
20170503,-0.005637,0.011225,-0.049408,-0.069822,-0.019517,-0.057451,-0.017162,-0.013408,-0.035379,0.005009,...,0.035464,-0.061670,-0.116677,0.025755,0.017915,-0.040529,-0.037061,-0.045260,-0.031050,-0.052526
20170504,0.008647,0.024790,-0.045152,-0.080454,-0.047912,-0.047729,-0.012925,-0.045922,-0.016502,0.000723,...,0.038719,-0.050292,-0.126928,0.029239,0.030259,-0.050401,-0.031111,-0.062009,-0.055990,-0.030355
20170505,0.029650,0.025362,-0.026835,-0.070643,-0.053107,-0.004268,-0.027992,-0.053341,-0.022091,0.017399,...,0.035087,-0.052918,-0.122614,0.062180,0.049734,-0.048412,-0.018459,-0.070290,-0.086487,-0.036765
20170508,0.021017,0.034281,-0.005983,-0.055006,-0.039849,-0.007500,-0.039059,-0.067527,0.011253,0.028046,...,0.012388,-0.043975,-0.049807,0.065397,0.021249,-0.051913,-0.048080,-0.054598,-0.077470,-0.038463


## create_information_report

- ` jaqs.research.signaldigger.digger.create_information_report() `

**简要描述：**

- ic分析
- 需要执行process_signal_before_analysis后才能执行该方法

**返回：**

ic分析结果，并将结果保存至预设目录

**示例：**

In [13]:
%matplotlib inline
sd.create_information_report()

Information Analysis
                ic
IC Mean     -0.022
IC Std.      0.228
t-stat(IC)  -0.978
p-value(IC)  0.331
IC Skew      0.052
IC Kurtosis -1.054
Ann. IR     -0.097
Figure saved: E:\Users\BurdenBear\Documents\PycharmProjects\JAQS-fxdayu\docs\_source\information_report.pdf


## ic_report_data

- ` jaqs.research.signaldigger.digger.ic_report_data `

**简要描述：**

- ic分析相关数据
- 需要执行create_information_report后该属性才有值

**返回：**

由'daily_ic'，'monthly_ic'所组成的字典(dict)

**示例：**

In [14]:
print(sd.ic_report_data.keys())
sd.ic_report_data["daily_ic"].head()

dict_keys(['daily_ic', 'monthly_ic'])


,ic
trade_date,
2017-05-03,-0.283985
2017-05-04,-0.317086
2017-05-05,-0.324241
2017-05-08,-0.239354
2017-05-09,-0.216610


## create_returns_report

- ` jaqs.research.signaldigger.digger.create_returns_report() `

**简要描述：**

- 收益分析
- 需要执行process_signal_before_analysis后才能执行该方法

**返回：**

收益分析结果，并将结果保存至预设目录

**示例：**

In [15]:
sd.create_returns_report()

Figure saved: E:\Users\BurdenBear\Documents\PycharmProjects\JAQS-fxdayu\docs\_source\returns_report.pdf


## returns_report_data

- ` jaqs.research.signaldigger.digger.returns_report_data `

**简要描述：**

- 收益分析相关数据
- 需要执行create_returns_report后该属性才有值

**返回：**

由'period_wise_quantile_ret'，'cum_quantile_ret','cum_long_ret','cum_short_ret', 'period_wise_tmb_ret', 'cum_tmb_ret'所组成的字典(dict)
分别对应：各quantile每日收益,quantile累积收益,多头组合（按因子值大小加权做多因子值为正的）累积收益,空头组合（按因子值大小加权做空因子值为负的）累积收益，多空组合（等权做多quantile最大组、做空quantile最小组）每日收益、多空组合（等权做多quantile最大组、做空quantile最小组）累积收益

**示例：**

In [16]:
print(sd.returns_report_data.keys())
sd.returns_report_data['cum_quantile_ret'].head()

dict_keys(['period_wise_quantile_ret', 'cum_quantile_ret', 'cum_long_ret', 'cum_short_ret', 'period_wise_tmb_ret', 'cum_tmb_ret'])


,1,2,3,4,5
trade_date,,,,,
2017-05-03,-0.000403,-0.001536,-0.002002,-0.001690,-0.010550
2017-05-04,-0.000896,-0.003169,-0.004793,-0.004698,-0.023262
2017-05-05,-0.001330,-0.005273,-0.007637,-0.008967,-0.034614
2017-05-08,-0.002298,-0.007992,-0.009366,-0.012054,-0.044011
2017-05-09,-0.003656,-0.010253,-0.010315,-0.015345,-0.051984


## create_full_report

- ` jaqs.research.signaldigger.digger.create_full_report() `

**简要描述：**

- 收益分析+ic分析
- 需要执行process_signal_before_analysis后才能执行该方法

**返回：**

综合分析结果，并将结果保存至预设目录

**示例：**

In [17]:
sd.create_full_report()



Value of signals of Different Quantiles Statistics
              min        max        mean         std  count    count %
quantile                                                              
1          0.0000    15.6558    9.272868    3.925101   6052  20.145130
2         13.9403    24.6232   18.820587    2.565347   6014  20.018641
3         21.2179    34.1752   27.605086    2.902044   6008  19.998669
4         30.3209    54.3519   39.938008    5.234493   6014  20.018641
5         46.6792  1908.3311  177.118206  275.505146   5954  19.818920
Figure saved: E:\Users\BurdenBear\Documents\PycharmProjects\JAQS-fxdayu\docs\_source\returns_report.pdf
Information Analysis
                ic
IC Mean     -0.022
IC Std.      0.228
t-stat(IC)  -0.978
p-value(IC)  0.331
IC Skew      0.052
IC Kurtosis -1.054
Ann. IR     -0.097
Figure saved: E:\Users\BurdenBear\Documents\PycharmProjects\JAQS-fxdayu\docs\_source\information_report.pdf


{'cum_long_ret':               return
 trade_date          
 20170503   -0.005028
 20170504   -0.011079
 20170505   -0.017490
 20170508   -0.023620
 20170509   -0.029382
 20170510   -0.030436
 20170511   -0.029884
 20170512   -0.027715
 20170515   -0.027956
 20170516   -0.033576
 20170517   -0.039766
 20170518   -0.046491
 20170519   -0.053255
 20170522   -0.057716
 20170523   -0.061340
 20170524   -0.063918
 20170525   -0.063850
 20170526   -0.062985
 20170531   -0.061300
 20170601   -0.056372
 20170602   -0.054407
 20170605   -0.056188
 20170606   -0.056413
 20170607   -0.055436
 20170608   -0.051492
 20170609   -0.046313
 20170612   -0.039790
 20170613   -0.035323
 20170614   -0.033818
 20170615   -0.037348
 ...              ...
 20170821    0.001017
 20170822   -0.002939
 20170823   -0.003304
 20170824   -0.000364
 20170825    0.005638
 20170828    0.012052
 20170829    0.016982
 20170830    0.021363
 20170831    0.022620
 20170901    0.020687
 20170904    0.019317
 20170905    0.0